In [233]:
import os
import PETs_Tool


folder_raw = '.\\data'
folder_SD  = '.\\data_dpsd'



dict_PETs_setting = {'adult' : {'filename'    : '[Adult]'
                               ,'read_params' : {'downcast'       : 'N' # 'Y'
                                                ,'label_encoding' : 'Y'}
                            #    ,'describe_params' : {'data': {'?'}}
                               }
                    }
dict_filename = {'adult': {'raw' : '[Adt Income] adult.csv'
                          }
                }
dict_filename = {key: {**value, 'params': dict_PETs_setting.get(key, {})} 
                 for key, value in dict_filename.items()}
# print(dict_filename)



# update default in SDV
list_adult_SD_SDV_train = ['CoupulaGAN'
                          ,'CTGAN'
                          ,'GaussianCoupula'
                          ,'TVAE'
                          ]
dict_adult_SD_SDV_train = {k: {'sd_params': {'model': k}} for k in list_adult_SD_SDV_train}
dict_filename = {key: PETs_Tool.PETs_util.update_append_nested(value
                                                              ,{'SD_train': dict_adult_SD_SDV_train}
                                                              ) 
                 for key, value in dict_filename.items()}
dict_adult_SD_SDV_train_default = {'sd_params': {'save_model': 'N'}
                                  ,'sample'   : 'Y'
                                  ,'sample_params' : {'sample_rows_as_raw' : 'Y'
                                                     ,'save_data' : 'N'}
                                  }
dict_adult_SD_train = {'SDV' : {key: PETs_Tool.PETs_util.update_append_nested(dict_adult_SD_SDV_train_default
                                                                       ,dict_adult_SD_SDV_train.get(key ,{})
                                                                       )
                                for key in dict_adult_SD_SDV_train
                               }
                      }
dict_filename = {k1: {k2: dict_adult_SD_train if k2 == 'SD_train' else v2
                      for k2 ,v2 in dict_filename[k1].items()
                     } 
                 for k1 in dict_filename
                }
# print(dict_filename)



dict_bootstrap = {'bootstrap_params' : {'bootstrap_time' : 3 # 30
                                       ,'sample_ratio'   : 0.8
                                       ,'random_state'   : None
                                       }
                 ,'index_params' : {'index_save'      : 'N'
                                   ,'index_save_data' : 'N'
                                   ,'index_filename'  : 'Anomymeter'
                                   }
                 }
# import json
# print(json.dumps(dict_bootstrap ,indent=4))

__Synther_param = {'sd_params': {'metadata' : 'SingleTable'
                                ,'save_model' : 'N'}
                  ,'sample' : 'Y'
                  ,'sample_params' : {'sample_rows_as_raw' : 'Y'
                                     ,'save_data' : 'N'}
                  }
aux_cols = [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
           ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
           ]

__param_anonymeter = {'SinglingOut': {'univariate'   : 'Y'
                                     ,'multivariate' : 'N'
                                     }
                     ,'Linkability' : 'Y'
                     ,'Linkability_params' : {'aux_cols' : aux_cols}
                     ,'Inference' : 'Y'
                     ,'Inference_params' : {'secret' : ''} # all columns
                     }



__dict_anonymeter_result = {}
for __dataset ,v in dict_filename.items():
    __filename = v['params']['filename']
    print(f"Now is {__filename}: {v['raw']} comparision.")

    __Loader = PETs_Tool.PETs_Loader(os.path.join(folder_raw ,v['raw'])
                                    ,v['params']
                                    )
    __df_ori = __Loader.data
    print(f"Now is {__filename} original data been load: {v['raw']}.")

    dict_boostrap_index = PETs_Tool.PETs_util.df_bootstrap(__df_ori ,dict_bootstrap)
    digits_max_boostrap = len(str(max(dict_boostrap_index.keys()))) # max boostrap times for fill zero

    for __library ,__dict_sd in v['SD_train'].items():
        print(f"Now is {__filename} by library {__library}.")

        for __method ,__params in __dict_sd.items():
            print(f"Now is {__filename} by method {__method} in library {__library}.")

            import copy
            for time ,dict_idx in dict_boostrap_index.items():
                __trail_name = f"{__filename}_{__library}_{__method}_{str(time).zfill(digits_max_boostrap)}"
                print(f"Trail as {__trail_name}.")

                ####### ####### #######
                # init - ori ,ctrl ,syn #
                ####### ####### #######
                __Synther = copy.deepcopy(__Loader)
                __data = __Synther.data.copy()
                __Synther.data_ori     = __data.loc[dict_boostrap_index[time]['idx_train'     ]].reset_index(drop=True)
                __Synther.data_control = __data.loc[dict_boostrap_index[time]['idx_validation']].reset_index(drop=True)
                __Synther.data         = __Synther.data_ori.copy()
                __Synther.data_syn     = PETs_Tool.PETs_SD_SDV(__Synther ,__Synther_param).synthetic_data
                __Synther.data = __data.copy()


                ####### ####### #######
                # _anonymeter         #
                ####### ####### #######
                __anonymeter = PETs_eval_anonymeter(data_ori     = __Synther.data_ori
                                                   ,data_syn     = __Synther.data_syn
                                                   ,data_control = __Synther.data_control
                                                   ,params = __param_anonymeter
                                                   )
                __evaluator = __anonymeter.evaluators
                
                __dict_anonymeter_result[__trail_name] = {}
                if 'SinglingOut' in __evaluator:
                    if 'univariate' in __evaluator['SinglingOut']:
                        __dict_anonymeter_result[__trail_name]['SinglingOut_univariate_Risk'] = __evaluator['SinglingOut']['univariate'].risk().value
                    if 'multivariate' in __evaluator['SinglingOut']:
                        __dict_anonymeter_result[__trail_name]['SinglingOut_multivariate_Risk'] = __evaluator['SinglingOut']['multivariate'].risk().value

                if 'Linkability' in __evaluator:
                    __dict_anonymeter_result[__trail_name]['Linkability_Risk'] = __evaluator['Linkability'].risk().value
                
                if 'Inference' in __evaluator:
                    __dict_anonymeter_result[__trail_name].update({'Inference_'+__secret: __secret_result.risk().value for __secret ,__secret_result in __evaluator['Inference'].items()})




                # 20231116, Justyn: refer to
                # https://github.com/statice/anonymeter/blob/main/notebooks/anonymeter_example.ipynb
                # https://storage.googleapis.com/statice-public/anonymeter-datasets



Now is [Adult]: [Adt Income] adult.csv comparision.
Now is [Adult] original data been load: [Adt Income] adult.csv.
Bootstrap df: 3 times is done.
Now is [Adult] by library SDV.
Now is [Adult] by method CoupulaGAN in library SDV.
Trail as [Adult]_SDV_CoupulaGAN_0.
Metafile loading time: 0.0312 sec.
We are execute SingleTable - GaussianCoupula.


In [231]:
__anonymeter.evaluators

{'SinglingOut': {},
 'Linkability': <anonymeter.evaluators.linkability_evaluator.LinkabilityEvaluator at 0x181de747c70>,
 'Inference': {'age': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181dda3ec50>,
  'workclass': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181c5a54fa0>,
  'fnlwgt': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181dba7f070>,
  'education': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181dda3e680>,
  'educational-num': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181dba7e020>,
  'marital-status': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181c5ce4130>,
  'occupation': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181de24eb30>,
  'relationship': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181de24fdc0>,
  'race': <anonymeter.evaluators.inference_evaluator.InferenceEvaluator at 0x181de24e8c0>,
 

In [230]:
__anonymeter = PETs_eval_anonymeter(data_ori     = __Synther.data_ori
                                                   ,data_syn     = __Synther.data_syn
                                                   ,data_control = __Synther.data_control
                                                   ,params = __param_anonymeter
                                                   )

Now is Singling-Out Evaluator


Found 203 failed queries out of 500. Check DEBUG messages for more details.


Singling out evaluation failed with Optimal parameters not found: The maximum number of function evaluations is exceeded..Please re-run this cell.For more stable results increase `n_attacks`. Note that this will make the evaluation slower.
Singling-Out Evaluator time: 34.2756 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 6.1276 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 2.0831 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 2.0747 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 2.0553 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 2.0643 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 2.3981 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 2.9898 s

In [229]:
class PETs_eval_anonymeter():
    import PETs_Tool
    from anonymeter import evaluators

    def __init__(self
                ,data_ori ,data_syn ,data_control
                ,params={} ,**kwargs):
        ####### ####### #######
        # init - params       #
        ####### ####### #######
        # {'evaluate_params' : {'anonymeter':
        default_params = {'SinglingOut' : {'univariate'   : 'Y'
                                          ,'multivariate' : 'N'
                                          }
                         ,'SinglingOut_params' : {'univariate'   : {'n_attacks' : 500}
                                                 ,'multivariate' : {'n_attacks' : 500
                                                                   ,'n_cols'    : 2
                                                                   }
                                                 }
                         ,'Linkability' : 'Y'
                         ,'Linkability_params' : {'n_attacks'   : 2000
                                                 ,'aux_cols'    : []
                                                 # [TODO] study how to set n_neighbors
                                                 ,'n_neighbors' : 10
                                                 # n_jobs follow joblib convention.
                                                 # -1 = all cores,
                                                 # -2 = all execept one
                                                 ,'n_jobs'      : -2
                                                 }
                         ,'Inference' : 'Y'
                         ,'Inference_params' : {'n_attacks'   : 1000
                                               ,'secret'      : ''
                                               # n_jobs follow joblib convention.
                                               # -1 = all cores,
                                               # -2 = all execept one
                                               ,'n_jobs'      : -2
                                               }
                         }
        self.params = update_append_nested(default_params ,params)
        self.data_ori     = data_ori
        self.data_syn     = data_syn
        self.data_control = data_control
        self.evaluators   = {}

        import warnings
        with warnings.catch_warnings():
            # anonymeter\evaluators\singling_out_evaluator.py:97:
            # FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.
            # Use isinstance(dtype, CategoricalDtype) instead elif is_categorical_dtype(values)
            warnings.simplefilter("ignore" ,category=FutureWarning)
            # c:\Program Files\Python310\lib\site-packages\anonymeter\stats\confidence.py:215:
            # UserWarning: Attack is as good or worse as baseline model.
            # Estimated rates: attack = 0.30674239114619767, baseline = 0.30773856438771213.
            # Analysis results cannot be trusted. self._sanity_check()
            warnings.simplefilter("ignore" ,category=UserWarning)

            ####### ####### #######
            # init - SinglingOut  #
            ####### ####### #######
            if any(v == 'Y' for v in self.params['SinglingOut'].values()):
                self.evaluators['SinglingOut'] = {}

                if self.params['SinglingOut']['univariate'] == 'Y':
                    self.__SinglingOutEvaluator(mode = 'univariate'
                                               ,**self.params['SinglingOut_params']['univariate']
                                               )
                # if self.params['SinglingOut']['multivariate'] == 'Y':
                #     self.__SinglingOutEvaluator(mode = 'multivariate'
                #                                ,**self.params['SinglingOut_params']['multivariate']
                #                                )
            ####### ####### #######
            # init - SinglingOut  #
            ####### ####### #######
            if self.params['Linkability'] == 'Y':
                self.__LinkabilityEvaluator(**self.params['Linkability_params'])


            if self.params['Inference'] == 'Y':
                self.evaluators['Inference'] = {}
                if self.params['Inference_params']['secret'] == '':
                    for __secret in self.data_syn.columns:
                        self.params['Inference_params']['secret'] = __secret
                        self.__InferenceEvaluator(**self.params['Inference_params'])
                else:
                    self.__InferenceEvaluator(**self.params['Inference_params'])




    ####### ####### #######
    # SinglingOut         #
    ####### ####### #######
    def __SinglingOutEvaluator(self ,mode ,**kwargs):
        import time
        __time_start = time.time()
        print('Now is Singling-Out Evaluator')
        __evaluator = evaluators.SinglingOutEvaluator(ori       = self.data_ori
                                                     ,syn       = self.data_syn
                                                     ,control   = self.data_control
                                                     ,**kwargs
                                                     )
        try:
            __evaluator.evaluate(mode=mode)
            # print(__evaluator.risk())
            # print("Successs rate of main attack    :" ,__evaluator.results().attack_rate  )
            # print("Successs rate of baseline attack:" ,__evaluator.results().baseline_rate)
            # print("Successs rate of control attack :" ,__evaluator.results().control_rate )
            self.evaluators['SinglingOut'][mode] = __evaluator
        except RuntimeError as ex:
            print(f"Singling out evaluation failed with {ex}."
                   "Please re-run this cell."
                   "For more stable results increase `n_attacks`. Note that this will "
                   "make the evaluation slower.")
        # multivariance
        # try:
        #     __evaluator.evaluate(mode='multivariate')
        #     __risk = __evaluator.risk()
        #     print(__risk)
        #     res = __evaluator.results()
        #     print(__evaluator.queries()[:3])
        # except RuntimeError as ex:
        #     print(f"Singling out evaluation failed with {ex}. Please re-run this cell."
        #           "For more stable results increase `n_attacks`. Note that this will "
        #           "make the evaluation slower.")
        print(f"Singling-Out Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Linkability         #
    ####### ####### #######
    def __LinkabilityEvaluator(self ,n_jobs ,aux_cols ,**kwargs):
        import time
        __time_start = time.time()
        print('Now is Linkability Evaluator')
        __str_aux_cols = "\nand ".join(f"[{', '.join(row)}]" for row in aux_cols)
        print(f"aux_cols are {__str_aux_cols}.")
        __evaluator = evaluators.LinkabilityEvaluator(ori      = self.data_ori
                                                     ,syn      = self.data_syn
                                                     ,control  = self.data_control
                                                     ,aux_cols = aux_cols
                                                     ,**kwargs
                                                     )
        __evaluator.evaluate(n_jobs=n_jobs)
        self.evaluators['Linkability'] = __evaluator
        # print(__evaluator.risk())
        # print("Successs rate of main attack    :" ,__evaluator.results().attack_rate  )
        # print("Successs rate of baseline attack:" ,__evaluator.results().baseline_rate)
        # print("Successs rate of control attack :" ,__evaluator.results().control_rate )
        print(f"Linkability Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Inference           #
    ####### ####### #######
    def __InferenceEvaluator(self ,secret ,n_jobs ,**kwargs):
        import time
        __time_start = time.time()
        print(f"Now is Inference Evaluator: secret is {secret}.")
        __aux_cols = [col for col in self.data_syn.columns if col != secret]
        __evaluator = evaluators.InferenceEvaluator(ori      = self.data_ori
                                                   ,syn      = self.data_syn
                                                   ,control  = self.data_control
                                                   ,aux_cols = __aux_cols
                                                   ,secret   = secret
                                                   ,**kwargs
                                                   )
        __evaluator.evaluate(n_jobs=n_jobs)
        # print(__evaluator.risk())
        # print("Successs rate of main attack    :" ,__evaluator.results().attack_rate  )
        # print("Successs rate of baseline attack:" ,__evaluator.results().baseline_rate)
        # print("Successs rate of control attack :" ,__evaluator.results().control_rate )
        self.evaluators['Inference'][secret] = __evaluator
        print(f"Inference Evaluator time: {round(time.time()-__time_start ,4)} sec.")
        

In [202]:

aux_cols = [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
           ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
           ]

__param_anonymeter = {'SinglingOut': {'univariate'   : 'Y'
                                     ,'multivariate' : 'N'
                                     }
                     ,'Linkability' : 'Y'
                     ,'Linkability_params' : {'aux_cols' : aux_cols}
                     ,'Inference' : 'Y'
                     ,'Inference_params' : {'secret' : ''} # all columns
                     }

__anonymeter = PETs_eval_anonymeter(data_ori     = __Synther.data_ori
                                   ,data_syn     = __Synther.data_syn
                                   ,data_control = __Synther.data_control
                                   ,params = __param_anonymeter
                                   )

Now is Singling-Out Evaluator


Found 200 failed queries out of 500. Check DEBUG messages for more details.


Singling-Out Evaluator time: 40.643 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 9.0073 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 3.2676 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 2.2787 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 2.2634 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 2.2382 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 2.4507 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 2.5689 sec.
Now is Inference Evaluator: secret is occupation.
Inference Evaluator time: 2.2586 sec.
Now is Inference Evaluator: secret is relationship.
Inference Evaluator time: 2.2576 sec.
Now is Inference Evaluator: secret is race.
Inference Evalu

- 這是已經有檔案的做法 (Unfinished)

In [ ]:
dict_adult_SD_SDV = {'CoupulaGAN'      : ['[Adt Income]_[SDV]_[CoupulaGAN]_20231106_113038_Asia_Taipei_10000.csv']
                    ,'CTGAN'           : ['[Adt Income]_[SDV]_[CTGAN]_20231106_093702_Asia_Taipei_10000.csv']
                    ,'GaussianCoupula' : ['[Adt Income]_[SDV]_[GaussianCoupula]_20231106_093622_Asia_Taipei_10000.csv']
                    ,'TVAE'            : ['[Adt Income]_[SDV]_[TVAE]_20231106_105252_Asia_Taipei_10000.csv']
                    }
dict_filename = {key: {**value ,'SDV': dict_adult_SD_SDV} 
                 for key, value in dict_filename.items()}



for __dataset ,v in dict_filename.items():
    __filename = v['params']['filename']
    print(f"Now is {__filename}: {v['raw']} comparision.")

    __df_ori = PETs_Tool.PETs_Loader(os.path.join(folder_raw ,v['raw'])
                                    ,v['params']
                                    ).data
    print(f"Now is {__filename} original data been load: {v['raw']}.")

    for __library ,__dict_sd in v['SD'].items():
        print(f"Now is {__filename} by library {__library}.")

        for __method ,__list_sd in __dict_sd.items():
            print(f"Now is {__filename} by method {__method} in library {__library}.")
            __list_sd = [__list_sd] if isinstance(__list_sd ,str) else __list_sd
            for __filename_sd in __list_sd:
                # 20231116, Justyn: 這裡是合成資料有落地
                #                   如果必要，可以直接調用 PETs_SD_* 結果的.synthetic_data
                __df_syn = PETs_Tool.PETs_Loader(os.path.join(folder_SD ,__filename_sd)
                                                ,v['params']
                                                ).data
                print(f"Now is {__filename} synthetic data been load: {__filename_sd}.")

                # 20231116, Justyn: refer to
                # https://github.com/statice/anonymeter/blob/main/notebooks/anonymeter_example.ipynb
                # https://storage.googleapis.com/statice-public/anonymeter-datasets

